# Load, normalize Spectra

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
%config InlineBackend.figure_format = "retina"
import matplotlib.pyplot as plt
from laspec.normalization import normalize_spectrum, normalize_spectra_block
from laspec.binning import rebin

In [2]:
import sys
sys.path.append('/home/jdli/jdli/')
mgiant_dir = '/share/jdli/MG/spec/'
# from mdwarf.preprocess import GroupSpec

from jdpy.group_specs import groupSpec
from jdpy.lm_download import LAMOST

obs_spec = groupSpec(root_dir=mgiant_dir)

In [4]:
# wave_interp = obs_spec[0]['wave']
wave = np.load('/share/jdli/spectra/wave_interp_R1800.npz')['wave']


In [ ]:
fluxes, invars = [], []
obsids = []
for i in tqdm(range(len(obs_spec))):
    flux_r, invar_r = rebin(obs_spec[i]['wave'], obs_spec[i]['flux'], 
                            obs_spec[i]['ivar'], wave_new=wave)

    fluxes += [flux_r]
    invars += [invar_r]
    obsids += [obs_spec[i]['obsid']]

In [8]:
fluxes, invars = np.array(fluxes), np.array(invars)

fluxes_norm, fluxes_cont = normalize_spectra_block(wave, fluxes, 
                                                   (6147., 8910.), 10., p=(1E-8, 1E-7), q=0.7, 
                                                   eps=1E-19, rsv_frac=2., n_jobs=64, verbose=5)

ivar_norm = fluxes_cont**2*invars

[Parallel(n_jobs=64)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  34 tasks      | elapsed:    2.3s
[Parallel(n_jobs=64)]: Done 160 tasks      | elapsed:    2.7s
[Parallel(n_jobs=64)]: Done 322 tasks      | elapsed:    3.0s
[Parallel(n_jobs=64)]: Done 520 tasks      | elapsed:    3.5s
[Parallel(n_jobs=64)]: Done 754 tasks      | elapsed:    4.0s
[Parallel(n_jobs=64)]: Done 1024 tasks      | elapsed:    4.5s
[Parallel(n_jobs=64)]: Done 1330 tasks      | elapsed:    5.2s
[Parallel(n_jobs=64)]: Done 1672 tasks      | elapsed:    5.9s
[Parallel(n_jobs=64)]: Done 2050 tasks      | elapsed:    6.6s
[Parallel(n_jobs=64)]: Done 2464 tasks      | elapsed:    7.5s
[Parallel(n_jobs=64)]: Done 2914 tasks      | elapsed:    8.5s
[Parallel(n_jobs=64)]: Done 3400 tasks      | elapsed:    9.5s
[Parallel(n_jobs=64)]: Done 3922 tasks      | elapsed:   10.5s
[Parallel(n_jobs=64)]: Done 4480 tasks      | elapsed:   11.7s
[Parallel(n_jobs=64)]: Done 5074 tasks      | 

In [ ]:
aspcap_raw = pd.read_csv('./data/mgiant_lm_ap16.csv')
print(aspcap_raw.shape, aspcap_raw.columns)
# %%
ind = (aspcap_raw['TEFF']>3400.) & (aspcap_raw['TEFF']<4300.) &\
        (np.abs(aspcap_raw['TEFF_ERR'])<100.) &\
        (np.abs(aspcap_raw['FE_H_ERR'])<0.1) & (aspcap_raw['FE_H']>-2.) &\
        (aspcap_raw['snri']>50.)

aspcap = aspcap_raw[ind]
print(aspcap.shape)
# aspcap['M_H'].plot(kind='hist')
# aspcap['M_H_ERR'].plot(kind='hist')
# aspcap['TEFF'].plot(kind='hist')
# aspcap['TEFF_ERR'].plot(kind='hist')


# aspcap.to_csv('./data/mgiant_lm_ap16_cut.csv', index=False)

# Save Spectra

In [26]:
def obsid_to_flux(all_obsid, all_flux, all_invar, newdf, wave):

    dr6_df = pd.DataFrame({
        'obsid':all_obsid
    })

    tr_df = pd.merge(newdf, dr6_df, on='obsid', how='left')
    print(tr_df.shape)

    obsids = np.array(all_obsid, dtype=int)
    tr_obsid = np.array(tr_df['obsid'].values, dtype=int)
    idx = np.searchsorted(obsids, tr_obsid)

    mask = np.isin(obsids, tr_obsid)  # np.in1d if np.isin is not available
    idx = np.where(mask)[0]

    tr_fluxes, tr_invars = all_flux[idx,:], all_invar[idx,:]
    tr_obsids = obsids[idx]
    print(tr_fluxes.shape, tr_invars.shape, tr_obsids.shape)

    new_tr_df = pd.DataFrame({'obsid':tr_obsids})
    tr_df = pd.merge(new_tr_df, tr_df, on='obsid', how='left')

    return {'new_df':tr_df, 'wave':wave, 'flux':tr_fluxes, 'invar':tr_invars}

ap_dict = obsid_to_flux(obsids, fluxes_norm, ivar_norm, aspcap, wave)

(2879, 23)
(2877, 1596) (2877, 1596) (2877,)


In [27]:
import pickle
save_ap_name = './data/mgaint_ap_train.pickcle'
with open(save_ap_name, "wb") as outfile:  
    pickle.dump(ap_dict, outfile, 
    # pickle.HIGHEST_PROTOCOL
    )

In [31]:
mgiant_spec_dict = {'obsid':np.array(obsids), 'wave':wave,
                    'flux_raw':fluxes, 'invar_raw':invars,
                    'flux':fluxes_norm, 'invar':ivar_norm,
                   }

with open('./data/mgiant_spec_dict.pickcle', "wb") as outfile:  
    pickle.dump(mgiant_spec_dict, outfile, 
    # pickle.HIGHEST_PROTOCOL
    )

In [30]:
mgiant_spec_dict['flux'].shape

(35335, 1596)